In [8]:
import numpy as np
import pygame
import tensorflow as tf

In [9]:
SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
BG_COLOR = (120, 120, 120)

DRAW_COLOR = (50, 50, 50)
DRAW_RADIUS = 2
ERASE_RADIUS = 5

CAR_WIDTH, CAR_HEIGHT = 50, 30
DEFAULT_START_X, DEFAULT_START_Y = 897, 379
DEFAULT_START_ANGLE = 290
DEFAULT_START_SPEED = 2

TRACK_SAVE_PATH = "track.png"
CAR_IMAGE_PATH = "Track_images/car.png"
TRACK_IMAGE_PATH = r"Track_images\silverstone_track.png"


FOR making th track drawing 
left ckick for drawing the track
right click for eraing the track

In [2]:
def draw_track():
    pygame.init()
    pygame.display.set_caption("Drawing the track, press 'S' to save as image.")
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    screen.fill(BG_COLOR)

    running, drawing, erase = True, False, False

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    drawing = True
                elif event.button == 3:
                    erase = True
            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:
                    drawing = False
                elif event.button == 3:
                    erase = False
            elif event.type == pygame.MOUSEMOTION:
                mouse_pos = pygame.mouse.get_pos()
                if drawing:
                    pygame.draw.circle(screen, DRAW_COLOR, mouse_pos, DRAW_RADIUS)
                elif erase:
                    pygame.draw.circle(screen, BG_COLOR, mouse_pos, ERASE_RADIUS)
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_s:
                pygame.image.save(screen, TRACK_SAVE_PATH)

        pygame.display.update()
    pygame.quit()


In [3]:
class Car:
    def __init__(self, image_path, x, y, angle=0, speed=0):
        image = pygame.image.load(image_path).convert_alpha()
        self.image = pygame.transform.scale(image, (CAR_WIDTH, CAR_HEIGHT))
        self.x, self.y = x, y
        self.angle = angle
        self.speed = speed

    def move(self):
        rad = np.radians(self.angle)
        self.x += self.speed * np.cos(rad)
        self.y -= self.speed * np.sin(rad)

    def draw(self, screen):
        rotated_img = pygame.transform.rotate(self.image, -self.angle)
        rect = rotated_img.get_rect(center=(self.x, self.y))
        screen.blit(rotated_img, rect.topleft)


In [4]:
def run_simulation(start_x=DEFAULT_START_X, 
                   start_y=DEFAULT_START_Y, 
                   start_angle=DEFAULT_START_ANGLE, 
                   start_speed=DEFAULT_START_SPEED):
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Car Simulation")

    track_image = pygame.image.load(TRACK_IMAGE_PATH).convert()
    car = Car(CAR_IMAGE_PATH, start_x, start_y, start_angle, start_speed)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        screen.fill(BG_COLOR)
        screen.blit(track_image, (0, 0))

        car.move()
        car.draw(screen)

        pygame.display.update()
        clock.tick(60)

    pygame.quit()


In [7]:
if __name__ == "__main__":
    # draw_track()
    run_simulation()
